<a href="https://colab.research.google.com/github/bhargavaCharyRudravelli/Bhargava_INFO5731_Fall2024/blob/main/Rudravelli_Bhargava_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [11]:
pip install requests beautifulsoup4 pandas


In [12]:
pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [26]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Get connection from the browser
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:123.0) Gecko/20100101 Firefox/123.0",
    "Accept": "text/html",
    "Connection": "keep-alive"
}

# Function to scrape reviews from the HTML content
def scrape_reviews(soup):
    review_containers = soup.find_all(class_='review-container')
    reviews = []
    for item in review_containers:
        username = item.find(class_='display-name-link').a.get_text() if item.find(class_='display-name-link') else "N/A"
        review = item.find(class_='text').get_text() if item.find(class_='text') else "N/A"
        rating_element = item.find(class_='rating-other-user-rating')
        rating = rating_element.get_text().strip() if rating_element else "N/A"
        reviews.append({
            'username': username,
            'review': review,
            'rating': rating
        })
    return reviews

# this function is used to load more reviews from the page
def get_more_reviews(pagination_key):
    ajax_url = f"https://www.imdb.com/title/tt0499549/reviews/_ajax?paginationKey={pagination_key}"
    response = requests.get(ajax_url, headers=headers)
    return BeautifulSoup(response.content, 'html.parser')

# Main function to scrape IMDb reviews, including dynamic loading of more reviews
def scrape_imdb_reviews(movie_url, number_of_reviews=1000):
    all_reviews = []
    # Initial request to get the first set of reviews
    response = requests.get(movie_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Scrape reviews from the initial page
    all_reviews.extend(scrape_reviews(soup))
    # Debug: Print how many reviews were scraped initially
    print(f"Initial reviews scraped: {len(all_reviews)}")
    # Loop to load more reviews using the AJAX pagination
    while len(all_reviews) < number_of_reviews:
        # Find the "Load More" button and check if it exists
        load_more_button = soup.find('div', class_='load-more-data')
        data_key = soup.find('div', class_='load-more-data') #this is needed to load more reviews from html
        if load_more_button:
            # Check if the button has the 'data-key' attribute before accessing it
            if 'data-key' in load_more_button.attrs:
                pagination_key = load_more_button['data-key']
                print("Pagination key found: {pagination_key}")

                # Get more reviews using AJAX
                soup = get_more_reviews(pagination_key)
                new_reviews = scrape_reviews(soup)
                print("more reviews scraped: {len(new_reviews)}")
                all_reviews.extend(new_reviews)
            else:
                print("No 'data-key' attribute found on the Load More button.")
                break
        else:
            # No more "Load More" button means no more reviews to load
            print("No 'Load More' button found.")
            break

        time.sleep(1)

    return all_reviews[:number_of_reviews]

# Scraping 1000 reviews from IMDb for the movie
movie_url = "https://www.imdb.com/title/tt0499549/reviews/?ref_=tt_ov_ql_2"
reviews_data = scrape_imdb_reviews(movie_url, number_of_reviews=1000)

# Check if reviews are being scraped
print(f"Total reviews scraped: {len(reviews_data)}")

# Save to CSV file if reviews are available
if reviews_data:
    with open('imdb_reviews.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['username', 'review', 'rating'])
        writer.writeheader()
        writer.writerows(reviews_data)
    print("CSV file 'imdb_reviews.csv' created successfully.")
else:
    print("No reviews were scraped.")


Initial reviews scraped: 25
Pagination key found: {pagination_key}
more reviews scraped: {len(new_reviews)}
Pagination key found: {pagination_key}
more reviews scraped: {len(new_reviews)}
Pagination key found: {pagination_key}
more reviews scraped: {len(new_reviews)}
Pagination key found: {pagination_key}
more reviews scraped: {len(new_reviews)}
Pagination key found: {pagination_key}
more reviews scraped: {len(new_reviews)}
Pagination key found: {pagination_key}
more reviews scraped: {len(new_reviews)}
Pagination key found: {pagination_key}
more reviews scraped: {len(new_reviews)}
Pagination key found: {pagination_key}
more reviews scraped: {len(new_reviews)}
Pagination key found: {pagination_key}
more reviews scraped: {len(new_reviews)}
Pagination key found: {pagination_key}
more reviews scraped: {len(new_reviews)}
Pagination key found: {pagination_key}
more reviews scraped: {len(new_reviews)}
Pagination key found: {pagination_key}
more reviews scraped: {len(new_reviews)}
Pagination k

# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [27]:
# Write code for each of the sub parts with proper comments.
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Load the CSV file with original reviews
df = pd.read_csv('imdb_reviews.csv')

# Initializing necessary components
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Function to clean each review
def clean_text(text):
    # Step 1: Remove special characters and punctuations
    text = re.sub(r'[^\w\s]', '', text)  # Removes special characters

    # Step 2: Remove numbers
    text = re.sub(r'\d+', '', text)  # Removes digits

    # Step 3: Tokenize the text and remove stopwords
    words = text.split()
    words = [word for word in words if word.lower() not in stop_words]

    # Step 4: Lowercase all texts
    words = [word.lower() for word in words]

    # Step 5: Stemming
    stemmed_words = [stemmer.stem(word) for word in words]

    # Step 6: Lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]

    # Return the cleaned text
    return ' '.join(lemmatized_words)

# Apply the clean_text function to the 'review' column
df['cleaned_review'] = df['review'].apply(clean_text)

# Saving the dataframe with the cleaned reviews into a new CSV file
df.to_csv('cleaned_imdb_reviews.csv', index=False)
print("Cleaned data saved in 'cleaned_imdb_reviews.csv'.")




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Cleaned data saved to 'cleaned_imdb_reviews.csv'.


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [28]:
# Your code here
import spacy
import pandas as pd
from collections import Counter

# Load the spacy English model
nlp = spacy.load('en_core_web_sm')

# Load the cleaned review CSV
df = pd.read_csv('cleaned_imdb_reviews.csv')

# Combine all cleaned reviews into one large text for analysis
text = " ".join(df['cleaned_review'].dropna().tolist())

# Step 1: POS Tagging
doc = nlp(text)
pos_counts = Counter()

for token in doc:
    if token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV']:  # Check for Nouns, Verbs, Adjectives, Adverbs
        pos_counts[token.pos_] += 1

print("POS Counts:")
print(f"Nouns (N): {pos_counts['NOUN']}")
print(f"Verbs (V): {pos_counts['VERB']}")
print(f"Adjectives (Adj): {pos_counts['ADJ']}")
print(f"Adverbs (Adv): {pos_counts['ADV']}")

# Step 2: Constituency and Dependency Parsing (using one sentence)
doc_sample = nlp(df['cleaned_review'].dropna().tolist()[0])

for sent in doc_sample.sents:
    print("Sentence:", sent.text)
    print("Dependency Parsing:")
    for token in sent:
        print(f"{token.text} ({token.dep_}) <-- {token.head.text}")
    break  # Show parsing for the first sentence only

# Step 3: Named Entity Recognition (NER)
ner_counts = Counter()

for ent in doc.ents:
    if ent.label_ in ['PERSON', 'ORG', 'GPE', 'PRODUCT', 'DATE']:  # Filter out specific types of entities
        ner_counts[ent.label_] += 1

print("Named Entity Counts:")
print("Person Names (PERSON): {ner_counts['PERSON']}")
print("Organizations (ORG): {ner_counts['ORG']}")
print("Locations (GPE): {ner_counts['GPE']}")
print("Products (PRODUCT): {ner_counts['PRODUCT']}")
print("Dates (DATE): {ner_counts['DATE']}")


POS Counts:
Nouns (N): 49848
Verbs (V): 23616
Adjectives (Adj): 19033
Adverbs (Adv): 5960
Sentence: rather intens privileg view jame cameron much anticip million budget return direct scene avatar empir leicest squar londonwher begin visual piec groundbreak termin seri titan one would expect cameron deliv visual far sharpest cgi seen could almost say disquiet follow cameron soul possibl strong intensifi qualiti product design visual effect noteworthi get prais upon offici releasewhat lack realli shape movi characterstori expect complex believ plot left movi mostli strong visual scifi lover desir mindbend philosophi fantasi explor limit outer speci factor would give voteavatar success cameron legaci intellig viral market avatar power market techniqu assembl success blockbust blair witch project rememb dark knight joker invad world also current product artific theartificecom intellig target marketkudo cameron avatar one movi year could get troubl share guy earli plea click ye comment use 

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [32]:
# Write your response below
This assignment took lot of time and at final I have completed it. As I am completely focused on all assignments and exercises.
I have learned many things from this assignment. I faced only the part where the reviews I can extract but not all.
So I tried to extract more reiews but it took me more time and finally from the html contents I undertood the way that
we can able to get load more data from the page and then save the data in csv. This part of the exercise took me more time to complete the assignment.